# Deepmatcher installation 

In [1]:
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher


     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 194kB 8.3MB/s 
  ERROR: Failed building wheel for fasttextmirror
    Running setup.py install for fasttextmirror ... done


In [2]:
import deepmatcher as dm
import torch
torch.cuda.is_available()
import pandas as pd
import io
from google.colab import files
from google.colab import drive


In [19]:
uploaded = files.upload()

Saving trainAMS.csv to trainAMS.csv
Saving validationAMS.csv to validationAMS.csv


# Data preprocessing


*   Tokenization
*   Data alteration
*   Word embeddings





In [4]:
trainRm, validationRm, testRm = dm.data.process( #fastText default embedding
    path='',
    train='trainRM.csv',
    validation='validationRM.csv',
    test='testRM.csv')


Reading and processing data from "trainRM.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "validationRM.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "testRM.csv"
0% [############################# ] 100% | ETA: 00:00:00INFO:deepmatcher.data.field:Downloading vectors from https://drive.google.com/uc?export=download&id=1Vih8gAmgBnuYDxfblbT94P6WjB7s1ZSh


downloading from Google Drive; may take a few minutes


wiki.en.bin: 8.49GB [02:16, 62.4MB/s]
INFO:deepmatcher.data.field:Extracting vectors into /root/.vector_cache

Building vocabulary
0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01

Computing principal components
0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


In [7]:
train_table = trainRm.get_raw_table()
train_table.head()

,id,label,left_id,left_name,left_host_id,left_host_name,left_neighbourhood_group,left_neighbourhood,left_latitude,left_longitude,left_room_type,left_price,left_minimum_nights,left_number_of_reviews,left_last_review,left_reviews_per_month,left_calculated_host_listings_count,left_availability_365,right_id,right_name,right_host_id,right_host_name,right_neighbourhood_group,right_neighbourhood,right_latitude,right_longitude,right_room_type,right_price,right_minimum_nights,right_number_of_reviews,right_last_review,right_reviews_per_month,right_calculated_host_listings_count,right_availability_365
0,35954,0,41319657,relax a domus nora garbatella/ ostiense,49813930,eleonora,,viii appia antica,41.86803,12.48814,private room,55,1,0,,,4,365,46499209,miniapartment with kitchen - fori imperiali,188811991,emanuele,,i centro storico,41.89427,12.48814,private room,71,1,0,,,4,244
1,2792,0,9613237,charming flat for 2/9 near vatican city,49002325,daniele,,i centro storico,41.91556,12.462760000000001,entire home/apt,370,2,22,2020-01-02,0.36,1,358,878455,between castel s. angel and s.peter,2484439,cristina,,i centro storico,41.9031,12.462760000000001,entire home/apt,43,2,106,2020-03-12,1.17,3,365
2,35002,0,34057550,guest house maison 6 la blanche,29638623,noemi,,x ostia/acilia,41.72976,12.27747,private room,89,1,6,2019-10-27,0.31,4,0,29797565,guest house maison 6 la dorée,29638623,noemi,,x ostia/acilia,41.72994,12.27747,private room,89,1,7,2020-07-18,0.35,4,365
3,37777,0,34273121,boutique station 's rooms 3,256169827,joseph,,i centro storico,41.89948,12.50188,private room,60,1,5,2019-09-18,0.25,3,365,44119503,doppia con letto aggiuntivo roma termini,353571465,severine,,i centro storico,41.89948,12.50188,private room,89,1,0,,,3,178
4,3869,0,11123872,elegant central apartment,55915005,pasquale,,i centro storico,41.90736,12.4967,entire home/apt,250,1,94,2020-01-02,1.65,5,180,1248462,colosseo - luxury apartment with terrace,6797602,maria alberta,,i centro storico,41.88808,12.4967,entire home/apt,117,1,15,2020-02-10,0.24,20,309


Let's print out the facts labeled with 1

In [8]:
    df = train_table[train_table["label"] == 1]
    df.head()

,id,label,left_id,left_name,left_host_id,left_host_name,left_neighbourhood_group,left_neighbourhood,left_latitude,left_longitude,left_room_type,left_price,left_minimum_nights,left_number_of_reviews,left_last_review,left_reviews_per_month,left_calculated_host_listings_count,left_availability_365,right_id,right_name,right_host_id,right_host_name,right_neighbourhood_group,right_neighbourhood,right_latitude,right_longitude,right_room_type,right_price,right_minimum_nights,right_number_of_reviews,right_last_review,right_reviews_per_month,right_calculated_host_listings_count,right_availability_365
7,2972,1,31512267,via degli scipioni 113 - studio,236062673,alberto,,i centro storico,41.90889,12.45887,entire home/apt,59,3,20,2020-10-15,0.9,3,1,39805212,via degli scipioni 113 - 1 bedroom apartment,236062673,alberto,,i centro storico,41.90889,12.45887,entire home/apt,65,3,1,2020-09-03,0.28,3,318
22,324,1,3602377,la piccola casa del capitano 33mq,14330516,carla,,i centro storico,41.9036,12.48813,entire home/apt,86,2,165,2020-02-17,2.41,5,92,4684890,la casa del capitano two rooms,14330516,carla,,i centro storico,41.9036,12.48813,entire home/apt,187,1,35,2019-11-04,0.49,5,79
25,3226,1,33217755,sallustiano guest house raspberry room x2 ! new,96122546,ilenia & fedor,,i centro storico,41.90562,12.49546,private room,51,2,0,,,106,0,33234338,guest house sallustiano yellow room x2 ! new,96122546,ilenia & fedor,,i centro storico,41.90562,12.49546,private room,48,2,15,2019-11-04,0.75,106,95
26,3836,1,38564409,rhome202 - traditional apartament,294239891,rhome,,vii san giovanni/cinecittà,41.87957,12.52144,entire home/apt,57,1,1,2019-11-04,0.07,2,251,40208447,rhome202 - a modern apartament,294239891,rhome,,vii san giovanni/cinecittà,41.87957,12.52144,entire home/apt,49,1,8,2020-10-19,0.61,2,342
31,836,1,35630140,al viminale hill inn & hotel - 2,20028020,38 viminale,,i centro storico,41.90025,12.49594,private room,43,1,1,2020-03-08,0.1,12,365,35633175,al viminale hill in & hotel - 9,20028020,38 viminale,,i centro storico,41.90025,12.49594,hotel room,47,1,15,2020-09-15,0.88,12,364


In [9]:
df.tail()

,id,label,left_id,left_name,left_host_id,left_host_name,left_neighbourhood_group,left_neighbourhood,left_latitude,left_longitude,left_room_type,left_price,left_minimum_nights,left_number_of_reviews,left_last_review,left_reviews_per_month,left_calculated_host_listings_count,left_availability_365,right_id,right_name,right_host_id,right_host_name,right_neighbourhood_group,right_neighbourhood,right_latitude,right_longitude,right_room_type,right_price,right_minimum_nights,right_number_of_reviews,right_last_review,right_reviews_per_month,right_calculated_host_listings_count,right_availability_365
11973,1490,1,19740759,appian way room- colosseum - / also ★long term★,112371287,pa dani & friends,,i centro storico,41.8922,12.50596,private room,85,2,36,2020-02-29,0.92,6,0,16869445,bigfamilyflats - coliseum & termini apt - 12pax,112371287,pa dani & friends,,i centro storico,41.8922,12.50596,entire home/apt,400,2,90,2020-01-21,1.93,6,261
11974,24,1,333368,apart trastevere-campo de ' fiori,1374312,giulio,,i centro storico,41.89189,12.46944,entire home/apt,210,1,45,2017-04-11,0.44,8,320,333369,magnificent apartment trastevere,1374312,giulio,,i centro storico,41.89189,12.46944,entire home/apt,213,1,7,2015-06-23,0.08,8,0
11977,1112,1,12801035,fellini - b & b roma città aperta,537257,alessandro lorenzo katia,,v prenestino/centocelle,41.8928,12.52735,hotel room,50,2,76,2019-08-23,1.39,4,166,13724634,rossellini - b & b roma città aperta,537257,alessandro lorenzo katia,,v prenestino/centocelle,41.8928,12.52735,hotel room,50,2,69,2019-09-02,1.3,4,166
11978,1752,1,19822145,in centro a via sistina nella casa di gogol,13261908,laura,,i centro storico,41.90435,12.48664,entire home/apt,150,4,0,,,14,0,19834621,attico spagna trinità dei monti,13261908,laura,,i centro storico,41.90435,12.48664,entire home/apt,200,4,1,2018-10-14,0.04,14,0
11979,4159,1,45950410,barberini-camera tripla con bagno privato interno,230950667,ancient rome,,i centro storico,41.90485,12.48726,private room,30,1,0,,,25,365,45950296,barberini-camera matrimoniale,230950667,ancient rome,,i centro storico,41.90485,12.48726,private room,40,1,0,,,25,365



# Model definition


1.   **Sif**: this model considers the alignment of sequences of words present in each attribute value pair to determine a match or non-match.
2.   **Rnn**: this model considers the words present in each attribute value pair to determine a match or non-match.
3.  **Attention mechanism**: this model considers the alignment of words present in each attribute value pair to determine a match or non-match. It does not take word order into account.
4.   **Hybrid**: this model considers the alignment of sequences of words present in each attribute value pair to determine a match or non-match.



In [10]:
model = dm.MatchingModel(attr_summarizer='hybrid') 
# model = dm.MatchingModel(attr_summarizer='sif') 
# model = dm.MatchingModel(attr_summarizer='rnn')
# model = dm.MatchingModel(attr_summarizer='attention') 

# Training
The model is trained on Rome's dataset

In [12]:
model.run_train(
    trainRm,
    validationRm,
    epochs=1,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 35333818
===>  TRAIN Epoch 1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2398: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:21:49


Finished Epoch 1 || Run Time: 1303.4 | Load Time:    7.7 || F1:  95.40 | Prec:  91.90 | Rec:  99.18 || Ex/s:   9.14

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:21


Finished Epoch 1 || Run Time:  138.3 | Load Time:    2.7 || F1:  99.48 | Prec:  98.97 | Rec: 100.00 || Ex/s:  28.33

* Best F1: tensor(99.4830)
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(99.4830)

In [14]:
# Compute F1 on test set
model.run_eval(testRm)

===>  EVAL Epoch 1


0% [█████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:48


Finished Epoch 1 || Run Time:  106.2 | Load Time:    2.4 || F1:  99.57 | Prec:  99.14 | Rec: 100.00 || Ex/s:  36.78



tensor(99.5687)

# Inference mode 
We infer the label over the Amsterdam's dataset

In [5]:
model = dm.MatchingModel(attr_summarizer='hybrid') 
model.load_state("hybrid_model.pth")

In [48]:
unlabeled = dm.data.process_unlabeled(
    path='trainAMS.csv',
    trained_model=model,
    ignore_columns = "label"
)


Reading and processing data from "trainAMS.csv"
0% [############################# ] 100% | ETA: 00:00:00

In [49]:
predictions = model.run_prediction(unlabeled)
predictions.head()

===>  PREDICT Epoch 1


0% [████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:28


Finished Epoch 1 || Run Time:   86.5 | Load Time:    1.9 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



,match_score
id,
695,0.981754
175,0.142853
329,0.983663
10207,0.144862
481,0.979656


In [50]:
predictions.tail()

,match_score
id,
6433,0.141631
2758,0.156833
14978,0.147905
3743,0.151321
8748,0.157153


In [51]:
predictions.insert(0, "ID", predictions.index)
predictions.head()

,ID,match_score
id,,
695,695,0.981754
175,175,0.142853
329,329,0.983663
10207,10207,0.144862
481,481,0.979656


In [52]:
matching_pairs = []

for pred in predictions.to_numpy():
  score = float(pred[1])
  if score > 0.90:
    matching_pairs.append(pred[0])
len(matching_pairs)


514

In [54]:
df = unlabeled.get_raw_table()
filtered_df = df[df.index.isin(matching_pairs)]
filtered = filtered_df[filtered_df["left_host_name"] == filtered_df["right_host_name"]]
filtered.head()

,id,left_id,left_name,left_host_id,left_host_name,left_neighbourhood_group,left_neighbourhood,left_latitude,left_longitude,left_room_type,left_price,left_minimum_nights,left_number_of_reviews,left_last_review,left_reviews_per_month,left_calculated_host_listings_count,left_availability_365,right_id,right_name,right_host_id,right_host_name,right_neighbourhood_group,right_neighbourhood,right_latitude,right_longitude,right_room_type,right_price,right_minimum_nights,right_number_of_reviews,right_last_review,right_reviews_per_month,right_calculated_host_listings_count,right_availability_365
2,329,12908561,"hotel in the heart of amsterdam , single room",67005410,feliciano,,centrum-west,52.37426,4.89926,private room,73,1,187,2020-02-12,3.36,16,365,12582071,hotel in the heart of amsterdam 3p,67005410,feliciano,,centrum-west,52.37426,4.89926,private room,153,1,143,2020-08-07,2.59,16,360
4,481,23899829,ph93 amsterdam - room p,8321212,henry,,centrum-west,52.3766,4.90207,private room,112,1,138,2020-09-14,4.16,2,294,24402987,ph93 amsterdam - room h,8321212,henry,,centrum-west,52.3766,4.90207,private room,136,1,37,2020-07-19,1.22,2,289
14,405,20913032,"hotel in the heart of amsterdam , single room",67005410,feliciano,,centrum-west,52.37426,4.89926,private room,73,1,50,2020-07-31,1.27,16,0,12865293,hotel in the heart of amsterdam 2p,67005410,feliciano,,centrum-west,52.37426,4.89926,private room,103,1,157,2020-08-09,2.81,16,365
22,830,45539926,stay friendly in our bunk room for 4 with en-s...,243878598,bunk,,oud-noord,52.3881,4.91438,private room,141,1,0,,,12,241,45539791,stay fabulous in our most epic room with en-suite,243878598,bunk,,oud-noord,52.3881,4.91438,private room,149,1,0,,,12,224
25,800,45539834,stay together in our bunk room for 3 with en-s...,243878598,bunk,,oud-noord,52.3881,4.91438,private room,106,1,0,,,12,244,45539425,stay hip in our semi-private bunk pod for 1,243878598,bunk,,oud-noord,52.3881,4.91438,shared room,30,1,0,,,12,241


In [55]:
filtered.tail()

,id,left_id,left_name,left_host_id,left_host_name,left_neighbourhood_group,left_neighbourhood,left_latitude,left_longitude,left_room_type,left_price,left_minimum_nights,left_number_of_reviews,left_last_review,left_reviews_per_month,left_calculated_host_listings_count,left_availability_365,right_id,right_name,right_host_id,right_host_name,right_neighbourhood_group,right_neighbourhood,right_latitude,right_longitude,right_room_type,right_price,right_minimum_nights,right_number_of_reviews,right_last_review,right_reviews_per_month,right_calculated_host_listings_count,right_availability_365
796,575,41709087,bed in 10 pod dorm ensuite,237150996,st christophers at the winston,,centrum-west,52.37392,4.89635,shared room,36,1,1,2020-02-02,0.09,10,251,41710302,private single ensuite,237150996,st christophers at the winston,,centrum-west,52.37392,4.89635,private room,95,1,0,,,10,0
801,715,45539466,stay cosy in our bunk room with en-suite,243878598,bunk,,oud-noord,52.3881,4.91438,private room,64,1,0,,,12,224,45539609,stay familiar in our bunk room for 4 with en-s...,243878598,bunk,,oud-noord,52.3881,4.91438,private room,141,1,0,,,12,244
813,687,45836338,the weber collection - luxurieus design studio...,178187873,marcel and pauline,,centrum-oost,52.36497,4.88061,entire home/apt,56,2,6,2020-11-17,3.21,25,0,45854688,the weber collection - luxurieus design studio...,178187873,marcel and pauline,,centrum-oost,52.36497,4.88061,entire home/apt,56,2,3,2020-11-04,1.8,25,0
821,462,22679718,your own private ship in amsterdam !,152013045,asile flottant,,oud-noord,52.39362,4.91033,entire home/apt,453,2,10,2020-02-05,0.33,6,0,21116197,your own luxurious private boat with cinema !,152013045,asile flottant,,oud-noord,52.39362,4.91033,entire home/apt,463,2,17,2020-07-05,0.44,6,0
822,299,12865495,hotel in the heart of amsterdam 2p,67005410,feliciano,,centrum-west,52.37426,4.89926,private room,104,1,164,2020-02-15,2.95,16,362,12582071,hotel in the heart of amsterdam 3p,67005410,feliciano,,centrum-west,52.37426,4.89926,private room,153,1,143,2020-08-07,2.59,16,360
